## WLAN V11 Sweep

### 0. Constants

In [1]:
PROJECT_NAME = "redfinch"  # decide with db to use
STDF_FILE_NAME = "sweep_with_svc.stdf.gz"  # should be in the same folder with this notebook
TEST_SUITE_NAME = "Main.WLAN.RFU_RX_char.a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20"  # target test suite

### 1. Upload stdf
Set connection string for uploading records.<br/>
<code>export DB_CONN_STR=mongodb://\<your_conn_str\> </code>


In [2]:
import os
from pymongo import MongoClient
from stdfparser import parse, StdfPub, StdfSubWlanSweep, DBConn

db_conn = DBConn("redfinch", os.path.join(os.path.abspath(""), STDF_FILE_NAME))

# parse
if db_conn.get_mir_id() is None:  # bypass stdf already in db
    pub = StdfPub(STDF_FILE_NAME)
    sub = StdfSubWlanSweep(db_conn)
    parse(pub, sub)

### 2. Fetch data

In [3]:
import pandas as pd
import numpy as np
from fetch import get_records_v11

df = pd.DataFrame.from_records(get_records_v11(db_conn, TEST_SUITE_NAME))
df.head()

,rssi_min,count,part_id,site,pout,v11
0,-39.3125,10,2,2,-40.0,1.065
1,-52.0625,10,2,2,-52.0,0.921
2,-82.5000,10,2,2,-82.0,0.936
3,-82.4375,10,2,2,-82.0,1.065
4,-58.0625,10,2,2,-58.0,1.036


### 3. Plot

In [4]:
from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

data = {}
for column in df.columns:
    data[column] = df[column].tolist()
    
ggplot(data=data) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="v11",  format=["V11: {} V"], ncol=2)

### SVC Offsets

In [5]:
from fetch import get_records_svc
svc_dict = get_records_svc(db_conn, text="svc1")
df["svc"] = [svc_dict.get(part_id, 0) for part_id in data["part_id"]]
df["svc_offset"] = df["v11"] - df["svc"]
df["svc_offset_index"] = df["svc_offset"] / 0.015
df = df.round({"svc_offset_index": 0})
df["svc_offset_index"] *= 15
df = df[df["svc_offset_index"] >= 0]
df.head()

,rssi_min,count,part_id,site,pout,v11,svc,svc_offset,svc_offset_index
0,-39.3125,10,2,2,-40.0,1.065,1.022,4.300004e-02,45.0
3,-82.4375,10,2,2,-82.0,1.065,1.022,4.300004e-02,45.0
4,-58.0625,10,2,2,-58.0,1.036,1.022,1.400004e-02,15.0
6,-45.7500,11,2,2,-46.0,1.022,1.022,4.482269e-08,0.0
8,-39.4375,10,2,2,-40.0,1.036,1.022,1.400004e-02,15.0


In [6]:
data = {}
for column in df.columns:
    data[column] = df[column].tolist()

ggplot(data=data) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="svc_offset_index",  format=["SVC Offset: {} mV"], ncol=2)